In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [2]:
# path
human_goslim_up_path = '../data/human_annotation/human_annotation_up.tsv'
rice_goslim_up_path = '../data/rice_annotation/rice_annotation_up.tsv'

human_goslim_all_path = '../data/biomart_goslim/biomart_human_goslim_R110.tsv'
rice_goslim_all_path = '../data/biomart_goslim/biomart_rice_goslim_R56.tsv'

In [3]:
human_goslim_up = pd.read_csv(human_goslim_up_path, sep='\t')
rice_goslim_up = pd.read_csv(rice_goslim_up_path, sep='\t')
human_goslim_all = pd.read_csv(human_goslim_all_path, sep='\t')
rice_goslim_all = pd.read_csv(rice_goslim_all_path, sep='\t')

human_goslim_all = human_goslim_all.drop_duplicates(subset=['Gene stable ID', 'GOSlim GOA Accession(s)', 'GOSlim GOA Description'], keep='first').copy()
rice_goslim_all = rice_goslim_all.drop_duplicates(subset=['Gene stable ID', 'GOSlim GOA Accession(s)', 'GOSlim GOA Description'], keep='first').copy()

display(human_goslim_up)
display(rice_goslim_up)
display(human_goslim_all)
display(rice_goslim_all)

/tmp/ipykernel_61298/269868789.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  human_goslim_all = pd.read_csv(human_goslim_all_path, sep='\t')
/tmp/ipykernel_61298/269868789.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  rice_goslim_all = pd.read_csv(rice_goslim_all_path, sep='\t')


,Gene stable ID,Chromosome/scaffold name,Gene start(bp),Gene end(bp),GOSlim GOA Accession(s),GOSlim GOA Description,HN-score(HN5),name
0,ENSG00000173110,1,161524540,161526894,GO:0003824,catalytic activity,245,HSPA6
1,ENSG00000173110,1,161524540,161526894,GO:0005576,extracellular region,245,HSPA6
2,ENSG00000173110,1,161524540,161526894,GO:0140657,ATP-dependent activity,245,HSPA6
3,ENSG00000173110,1,161524540,161526894,GO:0043226,organelle,245,HSPA6
4,ENSG00000173110,1,161524540,161526894,GO:0005856,cytoskeleton,245,HSPA6
...,...,...,...,...,...,...,...,...
1722,ENSG00000184205,X,53082367,53088540,GO:0005730,nucleolus,51,TSPYL2
1723,ENSG00000184205,X,53082367,53088540,GO:0023052,signaling,51,TSPYL2
1724,ENSG00000184205,X,53082367,53088540,GO:0042393,histone binding,51,TSPYL2
1725,ENSG00000184205,X,53082367,53088540,GO:0005694,chromosome,51,TSPYL2


,Gene stable ID,Chromosome/scaffold name,Gene start(bp),Gene end(bp),GOSlim GOA Accession(s),GOSlim GOA Description,HN-score(HN5)
0,Os04g0107900,4,483234,485978,GO:0016787,hydrolase activity,253
1,Os04g0107900,4,483234,485978,GO:0005886,plasma membrane,253
2,Os04g0107900,4,483234,485978,GO:0005515,protein binding,253
3,Os04g0107900,4,483234,485978,GO:0005488,binding,253
4,Os04g0107900,4,483234,485978,GO:0008150,biological_process,253
...,...,...,...,...,...,...,...
2522,Os01g0971800,1,42874273,42875515,GO:0008152,metabolic process,42
2523,Os01g0971800,1,42874273,42875515,GO:0009987,cellular process,42
2524,Os01g0971800,1,42874273,42875515,GO:0008150,biological_process,42
2525,Os01g0971800,1,42874273,42875515,GO:0005634,nucleus,42


,Gene stable ID,Chromosome/scaffold name,start2 (bp),end2 (bp),GOSlim GOA Accession(s),GOSlim GOA Description
0,ENSG00000243485,1,29554,31109,GO:0031047,gene silencing by RNA
1,ENSG00000284332,1,30366,30503,GO:0031047,gene silencing by RNA
2,ENSG00000186092,1,65419,71585,GO:0023052,signaling
3,ENSG00000186092,1,65419,71585,GO:0060089,molecular transducer activity
4,ENSG00000186092,1,65419,71585,GO:0005886,plasma membrane
...,...,...,...,...,...,...
581396,ENSG00000292372,Y,57207346,57212230,GO:0007010,cytoskeleton organization
581397,ENSG00000292372,Y,57207346,57212230,GO:0008092,cytoskeletal protein binding
581398,ENSG00000292372,Y,57207346,57212230,GO:0031410,cytoplasmic vesicle
581399,ENSG00000292372,Y,57207346,57212230,GO:0043226,organelle


,Gene stable ID,Chromosome/scaffold name,start1 (bp),end1 (bp),GOSlim GOA Accession(s),GOSlim GOA Description
0,Os01g0100100,1,2983,10815,GO:0006810,transport
1,Os01g0100100,1,2983,10815,GO:0008150,biological_process
2,Os01g0100100,1,2983,10815,GO:0009987,cellular process
3,Os01g0100100,1,2983,10815,GO:0003674,molecular_function
4,Os01g0100100,1,2983,10815,GO:0030234,enzyme regulator activity
...,...,...,...,...,...,...
258565,gene-rps19,Pt,134200,134481,GO:0003674,molecular_function
258566,gene-rps19,Pt,134200,134481,GO:0005198,structural molecule activity
258567,gene-rps19,Pt,134200,134481,GO:0005488,binding
258568,gene-rps19,Pt,134200,134481,GO:0003723,RNA binding


## Fold enrichment

1. GOSlimの数をカウントする （遺伝子とタームが1対1の関係になっているので実質遺伝子数のカウント）
2. Fold enrichmentを計算する


In [4]:
def count_goslim(df, file_name: str):
    unique_gene_term_pairs = df.drop_duplicates(subset=['Gene stable ID', 'GOSlim GOA Accession(s)']) # for debugging
    goslim_counts = unique_gene_term_pairs.groupby(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'])['Gene stable ID'].nunique().reset_index(name='counts')
    goslim_counts = goslim_counts.sort_values(by='counts', ascending=False).copy() # sort by counts
    results_directory = './goslim_counts'
    os.makedirs(results_directory, exist_ok=True)
    goslim_counts.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    return goslim_counts

human_goslim_up_counts = count_goslim(human_goslim_up, 'human_goslim_up_counts.tsv')
human_goslim_all_counts = count_goslim(human_goslim_all, 'human_goslim_all_counts.tsv')
rice_goslim_up_counts = count_goslim(rice_goslim_up, 'rice_goslim_up_counts.tsv')
rice_goslim_all_counts = count_goslim(rice_goslim_all, 'rice_goslim_all_counts.tsv')

display(human_goslim_up_counts)
display(human_goslim_all_counts)
display(rice_goslim_up_counts)
display(rice_goslim_all_counts)

,GOSlim GOA Accession(s),GOSlim GOA Description,counts
87,GO:0043226,organelle,128
15,GO:0005634,nucleus,71
72,GO:0023052,signaling,68
92,GO:0048856,anatomical structure development,63
13,GO:0005576,extracellular region,60
...,...,...,...
85,GO:0042254,ribosome biogenesis,1
78,GO:0031047,gene silencing by RNA,1
71,GO:0022600,digestive system process,1
59,GO:0009975,cyclase activity,1


,GOSlim GOA Accession(s),GOSlim GOA Description,counts
99,GO:0043226,organelle,18493
19,GO:0005634,nucleus,11009
83,GO:0023052,signaling,7321
36,GO:0005886,plasma membrane,6799
13,GO:0003824,catalytic activity,6372
...,...,...,...
117,GO:0071941,nitrogen cycle metabolic process,13
90,GO:0031386,protein tag,13
123,GO:0120274,virus coreceptor activity,1
66,GO:0007568,aging,1


,GOSlim GOA Accession(s),GOSlim GOA Description,counts
2,GO:0003674,molecular_function,204
12,GO:0005575,cellular_component,203
41,GO:0008150,biological_process,197
59,GO:0009987,cellular process,153
14,GO:0005622,intracellular anatomical structure,152
...,...,...,...
70,GO:0030154,cell differentiation,1
57,GO:0009856,pollination,1
74,GO:0040007,growth,1
76,GO:0045182,translation regulator activity,1


,GOSlim GOA Accession(s),GOSlim GOA Description,counts
2,GO:0003674,molecular_function,18866
16,GO:0005575,cellular_component,17584
51,GO:0008150,biological_process,16773
72,GO:0009987,cellular process,13761
14,GO:0005488,binding,12683
...,...,...,...
49,GO:0007623,circadian rhythm,38
47,GO:0007267,cell-cell signaling,12
69,GO:0009838,abscission,10
83,GO:0019825,oxygen binding,6


In [5]:
# 全ての遺伝子数を取得
human_unique_genes_all = human_goslim_all['Gene stable ID'].nunique()
rice_unique_genes_all = rice_goslim_all['Gene stable ID'].nunique()

# 抽出した遺伝子数を取得
human_unique_genes_up = pd.read_csv('../data/human_annotation/human_position_up.tsv', sep='\t')['Gene stable ID'].nunique()
rice_unique_genes_up = pd.read_csv('../data/rice_annotation/rice_position_up.tsv', sep='\t')['Gene stable ID'].nunique()

In [6]:
def fold_enrichment(df_goslim_up_counts, df_goslim_all_counts, unique_genes_up, unique_genes_all):
    
    merged_df = pd.merge(df_goslim_up_counts, 
                         df_goslim_all_counts,
                         on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], 
                         how='right', # if there is no match, fill with NaN or 0
                         suffixes=('_up', '_all'))
    
    merged_df['counts_up'] = merged_df['counts_up'].fillna(0)
    merged_df['counts_all'] = merged_df['counts_all'].fillna(0)
    merged_df['up_ratio'] = merged_df['counts_up'] / unique_genes_up
    merged_df['all_ratio'] = merged_df['counts_all'] / unique_genes_all

    merged_df['fold_enrichment'] = merged_df['up_ratio'] / merged_df['all_ratio']
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return merged_df


human_goslim_enrichment = fold_enrichment(human_goslim_up_counts, human_goslim_all_counts, human_unique_genes_up, human_unique_genes_all)
display(human_goslim_enrichment)

,GOSlim GOA Accession(s),GOSlim GOA Description,counts_up,counts_all,up_ratio,all_ratio,fold_enrichment
0,GO:0043226,organelle,128.0,18493,0.640,0.732135,0.874156
1,GO:0005634,nucleus,71.0,11009,0.355,0.435845,0.814510
2,GO:0023052,signaling,68.0,7321,0.340,0.289837,1.173072
3,GO:0005886,plasma membrane,52.0,6799,0.260,0.269171,0.965927
4,GO:0003824,catalytic activity,47.0,6372,0.235,0.252267,0.931554
...,...,...,...,...,...,...,...
132,GO:0071941,nitrogen cycle metabolic process,0.0,13,0.000,0.000515,0.000000
133,GO:0031386,protein tag,2.0,13,0.010,0.000515,19.430000
134,GO:0120274,virus coreceptor activity,0.0,1,0.000,0.000040,0.000000
135,GO:0007568,aging,0.0,1,0.000,0.000040,0.000000


In [ ]:
merged_df['goslim_counts_up'] = df_goslim_up_counts['counts_up'].fillna(0)
    merged_df['goslim_counts_all'] = df_goslim_all_counts['counts_all'].fillna(0)
    merged_df['up_ratio'] = (merged_df['counts_up'] / unique_genes_up)
    merged_df['all_ratio'] = (merged_df['counts_all'] / unique_genes_all)
    merged_df['fold_enrichment'] = (merged_df['counts_up'] / unique_genes_up) / (merged_df['counts_all'] / unique_genes_all)


human_goslim_enrichment = fold_enrichment(human_goslim_up_counts, human_goslim_all)
rice_goslim_enrichment = fold_enrichment(rice_goslim_up_counts, rice_goslim_all)

human_goslim_enrichment.to_csv(f'{results_directory}/human_goslim_enrichment.tsv', sep='\t', index=False)
rice_goslim_enrichment.to_csv(f'{results_directory}/rice_goslim_enrichment.tsv', sep='\t', index=False)

display(human_goslim_enrichment)
display(rice_goslim_enrichment)

## False discovery rate

1. calculate p-value
2. calculate FDR
3. calculate q-value

In [ ]:
def calculate_p_q_values(df_enrichment):
    """_summary_

    Args:
        df_enrichment (dataframe): _description_

    Returns:
        _type_: _description_
    """
    p_values = []
    df_enrichment['counts_up'] = df_enrichment['counts_up'].astype(int) # float(0.0) to int(0)
    total_all_genes = df_enrichment['counts_all'].sum()
    total_up_genes = df_enrichment['counts_up'].sum()

    for index, row in df_enrichment.iterrows():
        if row['counts_up'] == 0:
            p_values.append(1.0)
        else:
            observed_up = row['counts_up']
            observed_all = row['counts_all']
            observed_not_up = observed_all - observed_up
            total_not_up_genes = total_all_genes - total_up_genes

            _, p_value = fisher_exact([[observed_up, total_up_genes - observed_up], 
                                        [observed_not_up, total_not_up_genes - observed_not_up]], 
                                        alternative='greater')
            p_values.append(p_value)
    
    _, q_values, _, _ = multipletests(p_values, 
                                      alpha=0.05, 
                                      method='fdr_bh')
    
    df_enrichment['p_value'] = p_values
    df_enrichment['q_value'] = q_values

    return df_enrichment

In [ ]:
human_goslim_enrichment_with_p_q = calculate_p_q_values(human_goslim_enrichment)
rice_goslim_enrichment_with_p_q = calculate_p_q_values(rice_goslim_enrichment)

human_goslim_enrichment_with_p_q.to_csv(f'{results_directory}/human_goslim_enrichment_with_p_q.tsv', sep='\t', index=False)
rice_goslim_enrichment_with_p_q.to_csv(f'{results_directory}/rice_goslim_enrichment_with_p_q.tsv', sep='\t', index=False)

display(human_goslim_enrichment_with_p_q)
display(rice_goslim_enrichment_with_p_q)

In [ ]:
human_goslim_enrichment_with_p_q_up = human_goslim_enrichment_with_p_q[human_goslim_enrichment_with_p_q['counts_up'] > 0]
rice_goslim_enrichment_with_p_q_up = rice_goslim_enrichment_with_p_q[rice_goslim_enrichment_with_p_q['counts_up'] > 0]

common_enrichment = pd.merge(human_goslim_enrichment_with_p_q_up, rice_goslim_enrichment_with_p_q_up, 
                             on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], 
                             how='inner', 
                             suffixes=('_human', '_rice'))

display(human_goslim_enrichment_with_p_q_up)
display(rice_goslim_enrichment_with_p_q_up)
display(common_enrichment)

In [ ]:
common_goslim_path = '../data/common_term_up.tsv'
common_goslim_df = pd.read_csv(common_goslim_path, sep='\t')


human_common = human_goslim_enrichment_with_p_q_up[human_goslim_enrichment_with_p_q_up['GOSlim GOA Accession(s)'].isin(common_goslim_df['Unique GOSlim GOA Accession(s)'])]
human_common.sort_values(by='fold_enrichment', ascending=False, inplace=True)
human_common['GOSlim'] = human_common['GOSlim GOA Accession(s)'] + ': ' + human_common['GOSlim GOA Description']
cols = ['GOSlim'] + [col for col in human_common.columns if col != 'GOSlim']
human_common = human_common[cols]
human_common.drop(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], axis=1, inplace=True)

rice_common = rice_goslim_enrichment_with_p_q_up[rice_goslim_enrichment_with_p_q_up['GOSlim GOA Accession(s)'].isin(common_goslim_df['Unique GOSlim GOA Accession(s)'])]
rice_common.sort_values(by='fold_enrichment', ascending=False, inplace=True)
rice_common['GOSlim'] = rice_common['GOSlim GOA Accession(s)'] + ': ' + rice_common['GOSlim GOA Description']
cols = ['GOSlim'] + [col for col in rice_common.columns if col != 'GOSlim']
rice_common = rice_common[cols]
rice_common.drop(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], axis=1, inplace=True)

display(human_common)
display(rice_common)


## Visualization

In [ ]:
color = sns.color_palette("flare_r", as_cmap=True)

plt.figure(figsize=(6, 9))

dotplot = sns.scatterplot(
    data=human_common,
    x='fold_enrichment',
    y='GOSlim',
    size='counts_up', 
    hue='q_value',     
    palette=color, 
    legend=False
)

plt.grid(color='b', linestyle=':', linewidth=0.1)

plt.xlabel('Fold Enrichment')
plt.ylabel('Common GOSlim Terms')

plt.show()